# Interacting with Campaigns <a class="anchor" id="top"></a>

In this notebook, you will deploy and interact with campaigns in Amazon Personalize.

1. [Introduction](#intro)
1. [Create campaigns](#create)
1. [Interact with campaigns](#interact)
1. [Batch recommendations](#batch)
1. [Wrap up](#wrapup)

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have several solutions and at least one solution version for each. Once a solution version is created, it is possible to get recommendations from them, and to get a feel for their overall behavior.

This notebook starts off by deploying each of the solution versions from the previous notebook into individual campaigns. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 

As you with your customer on Amazon Personalize, you can modify the helper functions to fit the structure of their data input files to keep the additional rendering working.

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

![](img/05.png)

In [1]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random

import boto3
import pandas as pd

In [2]:
%store -r

In [3]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Interact with campaigns <a class="anchor" id="interact"></a>
[Back to top](#top)

Now that all campaigns are deployed and active, we can start to get recommendations via an API call. Each of the campaigns is based on a different recipe, which behave in slightly different ways because they serve different use cases. We will cover each campaign in a different order than used in previous notebooks, in order to deal with the possible complexities in ascending order (i.e. simplest first).

First, let's create a supporting function to help make sense of the results returned by a Personalize campaign. Personalize returns only an `item_id`. This is great for keeping data compact, but it means you need to query a database or lookup table to get a human-readable result for the notebooks. We will create a helper function to return a human-readable result from the LastFM dataset.

Start by loading in the dataset which we can use for our lookup table.

In [4]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(dataset_dir + '/movies.csv', sep=',', usecols=[0,1], encoding='latin-1', dtype={'movieId': "object", 'title': "str"},index_col=0)

# Render some sample data
items_df.head(5)

,title
movieId,
1,Toy Story (1995)
2,Jumanji (1995)
3,Grumpier Old Men (1995)
4,Waiting to Exhale (1995)
5,Father of the Bride Part II (1995)


By defining the ID column as the index column it is trivial to return an artist by just querying the ID.

In [5]:
movie_id_example = 589
title = items_df.loc[movie_id_example]['title']
print(title)

Terminator 2: Judgment Day (1991)


That isn't terrible, but it would get messy to repeat this everywhere in our code, so the function below will clean that up.

In [6]:
def get_movie_by_id(movie_id, movie_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return movie_df.loc[int(movie_id)]['title']
    except:
        return "Error obtaining title"

Now let's test a few simple values to check our error catching.

# A known good id (The Princess Bride)

In [7]:
print(get_movie_by_id(movie_id="1197"))

Princess Bride, The (1987)


# A bad type of value

In [8]:
print(get_movie_by_id(movie_id="987.9393939"))

Error obtaining title


# _The ^^ above ^^ cell should show an error_

In [9]:
# Really bad values
print(get_movie_by_id(movie_id="Steve"))

Error obtaining title


# _The ^^ above ^^ cell should show an error_

Great! Now we have a way of rendering results. 

### SIMS

SIMS requires just an item as input, and it will return items which users interact with in similar ways to their interaction with the input item. In this particular case the item is a movie. 

The cells below will handle getting recommendations from SIMS and rendering the results. Let's see what the recommendations are for the first item we looked at earlier in this notebook (Terminator 2: Judgment Day).

In [10]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = sims_campaign_arn,
    itemId = str(589),
)

In [11]:
item_list = get_recommendations_response['itemList']
for item in item_list:
    print(get_movie_by_id(movie_id=item['itemId']))

Jurassic Park (1993)
Braveheart (1995)
Terminator, The (1984)
Fugitive, The (1993)
Speed (1994)
Crimson Tide (1995)
GoldenEye (1995)
Batman (1989)
True Lies (1994)
Clear and Present Danger (1994)
Mask, The (1994)
Die Hard: With a Vengeance (1995)
In the Line of Fire (1993)
Lion King, The (1994)
Forrest Gump (1994)
Ghost (1990)
Apollo 13 (1995)
Star Trek: Generations (1994)
Cliffhanger (1993)
Firm, The (1993)
Die Hard (1988)
Seven (a.k.a. Se7en) (1995)
Indiana Jones and the Last Crusade (1989)
Mission: Impossible (1996)
Mrs. Doubtfire (1993)


Congrats, this is your first list of recommendations! This list is fine, but it would be better to see the recommendations for our sample collection of artists render in a nice dataframe. Again, let's create a helper function to achieve this.

In [12]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df(recommendations_df, movie_ID):
    # Get the movie name
    movie_name = get_movie_by_id(movie_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = sims_campaign_arn,
        itemId = str(movie_ID),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

Now, let's test the helper function with several different movies. Let's sample some data from our dataset to test our SIMS campaign. Grab 5 random movies from our dataframe.

Note: We are going to show similar titles, so you may want to re-run the sample until you recognize some of the movies listed

In [13]:
samples = items_df.sample(5)
samples

,title
movieId,
59258,Baby Mama (2008)
1942,All the King's Men (1949)
1946,Marty (1955)
84952,Confessions (Kokuhaku) (2010)
25886,Random Harvest (1942)


In [14]:
sims_recommendations_df = pd.DataFrame()
movies = samples.index.tolist()

for movie in movies:
    sims_recommendations_df = get_new_recommendations_df(sims_recommendations_df, movie)

sims_recommendations_df

,Baby Mama (2008),All the King's Men (1949),Marty (1955),Confessions (Kokuhaku) (2010),Random Harvest (1942)
0,Knight and Day (2010),Attack of the Giant Leeches (1959),"Ox-Bow Incident, The (1943)",Train to Busan (2016),High Sierra (1941)
1,Premonition (2007),"Asphyx, The (1973)","Dolce Vita, La (1960)",Take This Waltz (2011),Judgment at Nuremberg (1961)
2,It's Complicated (2009),7 Faces of Dr. Lao (1964),"Long, Hot Summer, The (1958)",B/W (2015),Wait Until Dark (1967)
3,"Duchess, The (2008)","Atomic Cafe, The (1982)",Von Ryan's Express (1965),War and Peace (2016),"Man Who Shot Liberty Valance, The (1962)"
4,Revolutionary Road (2008),"5,000 Fingers of Dr. T, The (1953)",Requiem for a Heavyweight (1962),Nymphomaniac: Volume I (2013),"Magnificent Seven, The (1960)"
5,Casanova (2005),Akira Kurosawa's Dreams (Dreams) (1990),"Suddenly, Last Summer (1959)",The Survivalist (2015),Escape from Alcatraz (1979)
6,Nanny McPhee (2005),Alexander Nevsky (Aleksandr Nevskiy) (1938),Kiss Me Deadly (1955),Scouts Guide to the Zombie Apocalypse (2015),Shall We Dance? (Shall We Dansu?) (1996)
7,August Rush (2007),After the Thin Man (1936),Who'll Stop the Rain (1978),Parasyte: Part 2 (2015),Forbidden Planet (1956)
8,Ju-on: The Grudge (2002),"Asphalt Jungle, The (1950)","Garden of the Finzi-Continis, The (Giardino de...",Parasyte: Part 1 (2014),Charade (1963)
9,Bride Wars (2009),20 Million Miles to Earth (1957),Happy Accidents (2000),Knockin' on Heaven's Door (1997),Mississippi Burning (1988)


You may notice that a lot of the items look the same, hopefully not all of them do (this is more likely with a smaller # of interactions). This shows that the evaluation metrics should not be the only thing you rely on when evaluating your solution version. So when this happens, what can you do to improve the results?

This is a good time to think about the hyperparameters of the Personalize recipes. The SIMS recipe has a `popularity_discount_factor` hyperparameter (see [documentation](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-sims.html)). Leveraging this hyperparameter allows you to control the nuance you see in the results. This parameter and its behavior will be unique to every dataset you encounter, and depends on the goals of the business. You can iterate on the value of this hyperparameter until you are satisfied with the results, or you can start by leveraging Personalize's hyperparameter optimization (HPO) feature. For more information on hyperparameters and HPO tuning, see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/customizing-solution-config-hpo.html).

### User Personalization

HRNN is one of the more advanced algorithms provided by Amazon Personalize. It supports personalization of the items for a specific user based on their past behavior and can intake real time events in order to alter recommendations for a user without retraining. 

Since HRNN relies on having a sampling of users, let's load the data we need for that and select 3 random users. Since Movielens does not include user data, we will select 3 random numbers from the range of user id's in the dataset.

In [15]:
if not USE_FULL_MOVIELENS:
    users = random.sample(range(1, 600), 3)
else:
    users = random.sample(range(1, 162000), 3)
users

[164, 563, 396]

Now we render the recommendations for our 3 random users from above. After that, we will explore real-time interactions before moving on to Personalized Ranking.

Again, we create a helper function to render the results in a nice dataframe.

#### API call results

In [16]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [17]:
recommendations_df_users = pd.DataFrame()
#users = users_df.sample(3).index.tolist()

for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

,164,563,396
0,Superman II (1980),Fargo (1996),Shrek (2001)
1,Star Trek II: The Wrath of Khan (1982),Yes Man (2008),"Amelie (Fabuleux destin d'AmÃ©lie Poulain, Le)..."
2,"Running Man, The (1987)","Holiday, The (2006)","Lord of the Rings: The Two Towers, The (2002)"
3,Star Trek V: The Final Frontier (1989),Borat: Cultural Learnings of America for Make ...,Toy Story 2 (1999)
4,Planet of the Apes (1968),"Devil Wears Prada, The (2006)",Good Will Hunting (1997)
5,Star Trek VI: The Undiscovered Country (1991),Notting Hill (1999),Eternal Sunshine of the Spotless Mind (2004)
6,"Terminator, The (1984)",The Imitation Game (2014),Spirited Away (Sen to Chihiro no kamikakushi) ...
7,Tron (1982),"Hurt Locker, The (2008)","Lord of the Rings: The Return of the King, The..."
8,Star Trek III: The Search for Spock (1984),Hitch (2005),Schindler's List (1993)
9,RoboCop (1987),Mean Girls (2004),LÃ©on: The Professional (a.k.a. The Profession...


Here we clearly see that the recommendations for each user are different. If you were to need a cache for these results, you could start by running the API calls through all your users and store the results, or you could use a batch export, which will be covered later in this notebook.

Now lets apply item filters to see recommendations for one of these users within a genre


In [18]:
def get_new_recommendations_df_by_filter(recommendations_df, user_id, filter_arn):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
        filterArn = filter_arn
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    filter_name = filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [19]:
user

396

You can see the recommendations for movies within a given genre. Within a VOD application you could create Shelves (also known as rails or carosels) easily by using these filters. Depending on the information you have about your items, You could also filter on additional information such as keyword, year/decade etc.

In [20]:
recommendations_df_shelves = pd.DataFrame()
for filter_arn in meta_filter_arns:
    recommendations_df_shelves = get_new_recommendations_df_by_filter(recommendations_df_shelves, user, filter_arn)
for filter_arn in decade_filter_arns:
    recommendations_df_shelves = get_new_recommendations_df_by_filter(recommendations_df_shelves, user, filter_arn)

recommendations_df_shelves

,Animation,Drama,Children,Crime,Film-Noir,Mystery,Sci-Fi,1970s
0,Shrek (2001),Good Will Hunting (1997),Shrek (2001),LÃ©on: The Professional (a.k.a. The Profession...,Mulholland Drive (2001),Donnie Darko (2001),Eternal Sunshine of the Spotless Mind (2004),"Godfather, The (1972)"
1,Toy Story 2 (1999),Eternal Sunshine of the Spotless Mind (2004),Toy Story 2 (1999),"Godfather, The (1972)",Chinatown (1974),"Sixth Sense, The (1999)",2001: A Space Odyssey (1968),One Flew Over the Cuckoo's Nest (1975)
2,Spirited Away (Sen to Chihiro no kamikakushi) ...,"Lord of the Rings: The Return of the King, The...","Monsters, Inc. (2001)",Pulp Fiction (1994),L.A. Confidential (1997),"Game, The (1997)",Donnie Darko (2001),Taxi Driver (1976)
3,"Monsters, Inc. (2001)",Schindler's List (1993),Finding Nemo (2003),City of God (Cidade de Deus) (2002),Notorious (1946),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),"Matrix Reloaded, The (2003)","Clockwork Orange, A (1971)"
4,Finding Nemo (2003),LÃ©on: The Professional (a.k.a. The Profession...,"Lion King, The (1994)",Fargo (1996),Sin City (2005),"City of Lost Children, The (CitÃ© des enfants ...",X-Men (2000),Monty Python and the Holy Grail (1975)
5,"Lion King, The (1994)","Beautiful Mind, A (2001)","Incredibles, The (2004)","O Brother, Where Art Thou? (2000)",Strangers on a Train (1951),Vertigo (1958),Star Wars: Episode VI - Return of the Jedi (1983),Star Wars: Episode IV - A New Hope (1977)
6,"Incredibles, The (2004)",Life Is Beautiful (La Vita Ã¨ bella) (1997),Shrek 2 (2004),"Green Mile, The (1999)",Dark City (1998),Who Framed Roger Rabbit? (1988),"Matrix Revolutions, The (2003)","Godfather: Part II, The (1974)"
7,Wallace & Gromit: The Best of Aardman Animatio...,Forrest Gump (1994),Toy Story (1995),Taxi Driver (1976),Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),Memento (2000),"Clockwork Orange, A (1971)",Apocalypse Now (1979)
8,Shrek 2 (2004),"Godfather, The (1972)",Beauty and the Beast (1991),Catch Me If You Can (2002),Miller's Crossing (1990),Reservoir Dogs (1992),Independence Day (a.k.a. ID4) (1996),Robin Hood (1973)
9,Toy Story (1995),"Lion King, The (1994)",Up (2009),Trainspotting (1996),Key Largo (1948),Minority Report (2002),Back to the Future (1985),Willy Wonka & the Chocolate Factory (1971)


The next topic is real-time events. Personalize has the ability to listen to events from your application in order to update the recommendations shown to the user. This is especially useful in media workloads, like video-on-demand, where a customer's intent may differ based on if they are watching with their children or on their own.

Additionally the events that are recorded via this system are stored until a delete call from you is issued, and they are used as historical data alongside the other interaction data you provided when you train your next models.

#### Real time events

Start by creating an event tracker that is attached to the campaign.

In [21]:
response = personalize.create_event_tracker(
    name='MovieTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']
event_tracker_arn = response['eventTrackerArn']

arn:aws:personalize:us-east-1:835319576252:event-tracker/1e2b508b
de689434-35d5-489a-9812-249066937d6a


We will create some code that simulates a user interacting with a particular item. After running this code, you will get recommendations that differ from the results above.

We start by creating some methods for the simulation of real time events.

In [22]:
session_dict = {}

def send_movie_click(USER_ID, ITEM_ID, EVENT_TYPE):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict[str(USER_ID)] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= str(USER_ID),
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': str(EVENT_TYPE),
        'properties': event_json
        }]
    )

def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id, event_type):
    # Get the artist name (header of column)
    movie_name = get_movie_by_id(item_id)
    # Interact with different movies
    print('sending event ' + event_type + ' for ' + get_movie_by_id(item_id))
    send_movie_click(USER_ID=user_id, ITEM_ID=item_id, EVENT_TYPE=event_type)
    # Get the recommendations (note you should have a base recommendation DF created before)
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        artist = get_movie_by_id(item['itemId'])
        recommendation_list.append(artist)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    #recommendations_df = recommendations_df.join(new_rec_DF)
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

At this point, we haven't generated any real-time events yet; we have only set up the code. To compare the recommendations before and after the real-time events, let's pick one user and generate the original recommendations for them.

In [23]:
# First pick a user
user_id = user

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    artist = get_movie_by_id(item['itemId'])
    recommendation_list.append(artist)
user_recommendations_df = pd.DataFrame(recommendation_list, columns = [user_id])
user_recommendations_df

,396
0,Shrek (2001)
1,"Amelie (Fabuleux destin d'AmÃ©lie Poulain, Le)..."
2,"Lord of the Rings: The Two Towers, The (2002)"
3,Toy Story 2 (1999)
4,Good Will Hunting (1997)
5,Eternal Sunshine of the Spotless Mind (2004)
6,Spirited Away (Sen to Chihiro no kamikakushi) ...
7,"Lord of the Rings: The Return of the King, The..."
8,Schindler's List (1993)
9,LÃ©on: The Professional (a.k.a. The Profession...


Ok, so now we have a list of recommendations for this user before we have applied any real-time events. Now let's pick 3 random artists which we will simulate our user interacting with, and then see how this changes the recommendations.

In [24]:
# Next generate 3 random movies
movies = items_df.sample(3).index.tolist()

In [25]:
# Note this will take about 15 seconds to complete due to the sleeps
for movie in movies:
    time.sleep(5)
    user_recommendations_df = get_new_recommendations_df_users_real_time(user_recommendations_df, user_id, movie,'click')
user_recommendations_df

sending event click for Pollyanna (1960)
sending event click for Somers Town (2008)
sending event click for Guest from the Future (Gostya iz buduschego) (1985)


,396,Pollyanna (1960),Somers Town (2008),Guest from the Future (Gostya iz buduschego) (1985)
0,Shrek (2001),Shrek (2001),American Graffiti (1973),Shrek (2001)
1,"Amelie (Fabuleux destin d'AmÃ©lie Poulain, Le)...","Amelie (Fabuleux destin d'AmÃ©lie Poulain, Le)...","Graduate, The (1967)","Amelie (Fabuleux destin d'AmÃ©lie Poulain, Le)..."
2,"Lord of the Rings: The Two Towers, The (2002)","Lord of the Rings: The Two Towers, The (2002)",Miracle on 34th Street (1947),"Lord of the Rings: The Two Towers, The (2002)"
3,Toy Story 2 (1999),Toy Story 2 (1999),"Producers, The (1968)",Toy Story 2 (1999)
4,Good Will Hunting (1997),Good Will Hunting (1997),Willy Wonka & the Chocolate Factory (1971),Good Will Hunting (1997)
5,Eternal Sunshine of the Spotless Mind (2004),Eternal Sunshine of the Spotless Mind (2004),"Charlie Brown Christmas, A (1965)",Eternal Sunshine of the Spotless Mind (2004)
6,Spirited Away (Sen to Chihiro no kamikakushi) ...,Spirited Away (Sen to Chihiro no kamikakushi) ...,Monty Python's And Now for Something Completel...,Spirited Away (Sen to Chihiro no kamikakushi) ...
7,"Lord of the Rings: The Return of the King, The...","Lord of the Rings: The Return of the King, The...","Apartment, The (1960)","Lord of the Rings: The Return of the King, The..."
8,Schindler's List (1993),Schindler's List (1993),Sleeper (1973),Schindler's List (1993)
9,LÃ©on: The Professional (a.k.a. The Profession...,LÃ©on: The Professional (a.k.a. The Profession...,"Philadelphia Story, The (1940)",LÃ©on: The Professional (a.k.a. The Profession...


In the cell above, the first column after the index is the user's default recommendations from HRNN, and each column after that has a header of the artist that they interacted with via a real time event, and the recommendations after this event occurred. 

The behavior may not shift very much; this is due to the relatively limited nature of this dataset and effect of a few random clicks. If you wanted to better understand this, try simulating clicking more movies, and you should see a more pronounced impact.

Now lets look at the event filters, which allow you to filter items based on the interaction data. For this dataset, it could be click or watch based on the data we imported, but could be based on whatever interaction schema you design (click, rate, like, watch, purchase etc.) For VOD shelves you could move a title from "Top picks for you" to a "Watch again", the watch again recommendations will be based on the users current interactions, but only recommend titles that have already been watched.


In [26]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

,watched,unwatched
0,LÃ©on: The Professional (a.k.a. The Profession...,Inglourious Basterds (2009)
1,NaN,Up (2009)
2,NaN,"Dark Knight, The (2008)"
3,NaN,Eternal Sunshine of the Spotless Mind (2004)
4,NaN,Forrest Gump (1994)
5,NaN,Little Miss Sunshine (2006)
6,NaN,WALLÂ·E (2008)
7,NaN,Thank You for Smoking (2006)
8,NaN,Children of Men (2006)
9,NaN,V for Vendetta (2006)


now lets send a watch event in for 4 unwatched recommendations, which would simulate watching 4 movies. In a VOD application, you may choose to send in an event after they have watched a significant amount (over 75%) of a piece of content. Sending at 100% complete could miss people that stop short of the credits.

In [27]:
 # Get the recommendations
top_unwatched_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = userpersonalization_campaign_arn,
    userId = str(user_id),
    filterArn = filter_arn,
    numResults=4)
item_list = top_unwatched_recommendations_response['itemList']
for item in item_list:
    print('sending event watch for ' + get_movie_by_id(item['itemId']))
    send_movie_click(USER_ID=user_id, ITEM_ID=item['itemId'], EVENT_TYPE='watch')
    time.sleep(10)


sending event watch for Inglourious Basterds (2009)
sending event watch for Up (2009)
sending event watch for Dark Knight, The (2008)
sending event watch for Eternal Sunshine of the Spotless Mind (2004)


Now we can look at the event filters to see the updated watched and unwatched recommendations 

In [28]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

,watched,unwatched
0,Eternal Sunshine of the Spotless Mind (2004),Shrek (2001)
1,NaN,"Amelie (Fabuleux destin d'AmÃ©lie Poulain, Le)..."
2,NaN,"Lord of the Rings: The Two Towers, The (2002)"
3,NaN,Good Will Hunting (1997)
4,NaN,Eternal Sunshine of the Spotless Mind (2004)
5,NaN,Spirited Away (Sen to Chihiro no kamikakushi) ...
6,NaN,"Lord of the Rings: The Return of the King, The..."
7,NaN,Schindler's List (1993)
8,NaN,"Beautiful Mind, A (2001)"
9,NaN,Life Is Beautiful (La Vita Ã¨ bella) (1997)


### Personalized Ranking

The core use case for personalized ranking is to take a collection of items and to render them in priority or probable order of interest for a user. For a VOD application you want dynamically render a personalized shelf/rail/carousel based on some information (director, location, superhero franchise, movie time period etc). This may not be information that you have in your metadata, so a item metadata filter will not work, howeverr you may have this information within you system to generate the item list. 

To demonstrate this, we will use the same user from before and a random collection of items.

In [29]:
rerank_user = user
rerank_items = items_df.sample(25).index.tolist()

Now build a nice dataframe that shows the input data.

In [30]:
rerank_list = []
for item in rerank_items:
    movie = get_movie_by_id(item)
    rerank_list.append(movie)
rerank_df = pd.DataFrame(rerank_list, columns = ['Un-Ranked'])
rerank_df

,Un-Ranked
0,"Flintstones, The (1994)"
1,Mystery Train (1989)
2,Cashback (2004)
3,Day Watch (Dnevnoy dozor) (2006)
4,Big (1988)
5,Fire and Ice (1983)
6,Life Stinks (1991)
7,Savages (2012)
8,"Barefoot Contessa, The (1954)"
9,Devil Dog: The Hound of Hell (1978)


Then make the personalized ranking API call.

In [31]:
# Convert user to string:
user_id = str(rerank_user)
rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = rerank_campaign_arn,
        userId = user_id,
        inputList = rerank_item_list
)

get_recommendations_response_rerank

{'ResponseMetadata': {'RequestId': '90792154-2717-4e16-a595-ec1b2703e8a7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'date': 'Sun, 08 Nov 2020 20:37:34 GMT',
   'x-amzn-requestid': '90792154-2717-4e16-a595-ec1b2703e8a7',
   'content-length': '1417',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'personalizedRanking': [{'itemId': '355', 'score': 0.2689658},
  {'itemId': '2797', 'score': 0.2234265},
  {'itemId': '80586', 'score': 0.0924742},
  {'itemId': '6203', 'score': 0.086949},
  {'itemId': '379', 'score': 0.0706614},
  {'itemId': '2691', 'score': 0.0666595},
  {'itemId': '168248', 'score': 0.0433316},
  {'itemId': '1892', 'score': 0.0300974},
  {'itemId': '72692', 'score': 0.0287358},
  {'itemId': '55207', 'score': 0.0266975},
  {'itemId': '148982', 'score': 0.023446},
  {'itemId': '3521', 'score': 0.0224833},
  {'itemId': '3546', 'score': 0.0121619},
  {'itemId': '4420', 'score': 0.0039101},
  {'itemId': '52950'},
  {'itemId': '5041

Now add the reranked items as a second column to the original dataframe, for a side-by-side comparison.

In [32]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    movie = get_movie_by_id(item['itemId'])
    ranked_list.append(movie)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df

,Un-Ranked,Re-Ranked
0,"Flintstones, The (1994)","Flintstones, The (1994)"
1,Mystery Train (1989),Big (1988)
2,Cashback (2004),Flipped (2010)
3,Day Watch (Dnevnoy dozor) (2006),Life Stinks (1991)
4,Big (1988),Timecop (1994)
5,Fire and Ice (1983),"Legend of 1900, The (a.k.a. The Legend of the ..."
6,Life Stinks (1991),John Wick: Chapter Two (2017)
7,Savages (2012),"Perfect Murder, A (1998)"
8,"Barefoot Contessa, The (1954)",Mickey's Once Upon a Christmas (1999)
9,Devil Dog: The Hound of Hell (1978),Cashback (2004)


You can see above how each entry was re-ordered based on the model's understanding of the user. This is a popular task when you have a collection of items to surface a user, a list of promotions for example.

## Batch recommendations <a class="anchor" id="batch"></a>
[Back to top](#top)

There are many cases where you may want to have a larger dataset of exported recommendations. Recently, Amazon Personalize launched batch recommendations as a way to export a collection of recommendations to S3. In this example, we will walk through how to do this for the HRNN solution. For more information about batch recommendations, please see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/getting-recommendations.html#recommendations-batch). This feature applies to all recipes, but the output format will vary.

A simple implementation looks like this:

```python
import boto3

personalize_rec = boto3.client(service_name='personalize')

personalize_rec.create_batch_inference_job (
    solutionVersionArn = "Solution version ARN",
    jobName = "Batch job name",
    roleArn = "IAM role ARN",
    jobInput = 
       {"s3DataSource": {"path": S3 input path}},
    jobOutput = 
       {"s3DataDestination": {"path":S3 output path"}}
)
```

The SDK import, the solution version arn, and role arns have all been determined. This just leaves an input, an output, and a job name to be defined.

Starting with the input for HRNN, it looks like:


```JSON
{"userId": "4638"}
{"userId": "663"}
{"userId": "3384"}
```

This should yield an output that looks like this:

```JSON
{"input":{"userId":"4638"}, "output": {"recommendedItems": ["296", "1", "260", "318"]}}
{"input":{"userId":"663"}, "output": {"recommendedItems": ["1393", "3793", "2701", "3826"]}}
{"input":{"userId":"3384"}, "output": {"recommendedItems": ["8368", "5989", "40815", "48780"]}}
```

The output is a JSON Lines file. It consists of individual JSON objects, one per line. So we will need to put in more work later to digest the results in this format.

### Building the input file

When you are using the batch feature, you specify the users that you'd like to receive recommendations for when the job has completed. The cell below will again select a few random users and will then build the file and save it to disk. From there, you will upload it to S3 to use in the API call later.

In [33]:
# We will use the same users from before
users
# Write the file to disk
json_input_filename = "json_input.json"
with open(data_dir + "/" + json_input_filename, 'w') as json_input:
    for user_id in users:
        json_input.write('{"userId": "' + str(user_id) + '"}\n')

In [34]:
# Showcase the input file:
!cat $data_dir"/"$json_input_filename

{"userId": "164"}
{"userId": "563"}
{"userId": "396"}


Upload the file to S3 and save the path as a variable for later.

In [35]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(data_dir+"/"+json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

s3://835319576252personalizepocvod/json_input.json


Batch recommendations read the input from the file we've uploaded to S3. Similarly, batch recommendations will save the output to file in S3. So we define the output path where the results should be saved.

In [36]:
# Define the output path
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

s3://835319576252personalizepocvod/


Now just make the call to kick off the batch export process.

In [37]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = userpersonalization_solution_version_arn,
    jobName = "VOD-POC-Batch-Inference-Job-UserPersonalization_" + str(round(time.time()*1000)),
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

Run the while loop below to track the status of the batch recommendation call. This can take around 30 minutes to complete, because Personalize needs to stand up the infrastructure to perform the task. We are testing the feature with a dataset of only 3 users, which is not an efficient use of this mechanism. Normally, you would only use this feature for bulk processing, in which case the efficiencies will become clear.

In [38]:
%%time

current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

Import Started on:  08:37:36 PM
DatasetInferenceJob: CREATE PENDING
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInfer

In [39]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, data_dir+"/"+export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(data_dir+"/"+export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the artists
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            movie = get_movie_by_id(item)
            recommendation_list.append(movie)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

,User: 164,User: 563,User: 396
0,Superman II (1980),Fargo (1996),Shrek (2001)
1,Star Trek II: The Wrath of Khan (1982),Yes Man (2008),"Amelie (Fabuleux destin d'AmÃ©lie Poulain, Le)..."
2,"Running Man, The (1987)","Holiday, The (2006)","Lord of the Rings: The Two Towers, The (2002)"
3,Star Trek V: The Final Frontier (1989),Borat: Cultural Learnings of America for Make ...,Toy Story 2 (1999)
4,Planet of the Apes (1968),"Devil Wears Prada, The (2006)",Good Will Hunting (1997)
5,Star Trek VI: The Undiscovered Country (1991),Notting Hill (1999),Eternal Sunshine of the Spotless Mind (2004)
6,"Terminator, The (1984)",The Imitation Game (2014),Spirited Away (Sen to Chihiro no kamikakushi) ...
7,Tron (1982),"Hurt Locker, The (2008)","Lord of the Rings: The Return of the King, The..."
8,Star Trek III: The Search for Spock (1984),Hitch (2005),Schindler's List (1993)
9,RoboCop (1987),Mean Girls (2004),LÃ©on: The Professional (a.k.a. The Profession...


## Wrap up <a class="anchor" id="wrapup"></a>
[Back to top](#top)

With that you now have a fully working collection of models to tackle various recommendation and personalization scenarios, as well as the skills to manipulate customer data to better integrate with the service, and a knowledge of how to do all this over APIs and by leveraging open source data science tools.

Use these notebooks as a guide to getting started with your customers for POCs. As you find missing components, or discover new approaches, cut a pull request and provide any additional helpful components that may be missing from this collection.

You'll want to make sure that you clean up all of the resources deployed during this POC. We have provided a separate notebook which shows you how to identify and delete the resources in `04_Clean_Up_Resources.ipynb`.

In [40]:
%store event_tracker_arn
%store batchInferenceJobArn

Stored 'event_tracker_arn' (str)
Stored 'batchInferenceJobArn' (str)


# Release Resources

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();